In [1]:
from csv import DictReader
import pandas as pd
import numpy as np

import pprint
pp = pprint.PrettyPrinter(indent=2)


"""
    Reads the GSML CSV file and creates a list where each row is in turn a list of;
    - document ID (adjusted for standard prefix of S and numbering from 61 to 90)
    - system name (simple form)
    - experiment ID (A, B, C)
    - error type (one of 6 types)
"""

# Convert the test set prefixes and numbering to get sequential IDs
test_set_ids = {'T'+str(i).zfill(3)+'.txt':'S'+str(i+60).zfill(3)+'.txt' for i in range(1,31)}

def format_doc_id(doc_id):
  d = doc_id
  if doc_id in test_set_ids:
    d = test_set_ids[doc_id]
  return d.replace('.txt','')

# GSML partition names to experiment IDs
def format_exp_id(exp_id):
  return {
    '00_Existing':  'A',
    '01_To_60':     'B',
    '02_To_90':     'C',
  }[exp_id]

# Use a simpler system name than the filenames
systems_mapping = {
    'wiseman_frototest_conditional_copy_beam5_gens.txt':  'conditional_copy',
    'puduppully_aaai_19_rotowire_test.txt':               'document_plan',
    'rebuffel_test_gen_not_paper.txt':                    'hierarchical_encoder',
  }

def format_sys_id(sys_filename):
  return systems_mapping[sys_filename]

# Read the docs file to get information on each document
with open('docs.csv', 'r') as fh:
  rows = list(DictReader(fh))
  doc_experiments = {format_doc_id(row['DOC_ID']):format_exp_id(row['EXP_ID']) for row in rows}
  doc_systems = {format_doc_id(row['DOC_ID']):format_sys_id(row['SYS_ID']) for row in rows}

# Read the list of errors and store as per above
results = []
with open('errors.csv', 'r') as fh:
  reader = DictReader(fh)

  for row in reader:
    doc_id = format_doc_id(row['TEXT_ID'])
    sys_id = doc_systems[doc_id]
    exp_id = doc_experiments[doc_id]
    typ_id = row['TYPE']
    results.append([doc_id,sys_id,exp_id,typ_id])


In [2]:
from collections import OrderedDict

# Dictionary to store measurements for the CV* code
measurements = OrderedDict()

systems = ['conditional_copy', 'document_plan', 'hierarchical_encoder']
experiments = ['A', 'B', 'C']
error_types = ['NAME', 'NUMBER', 'WORD', 'CONTEXT', 'NOT_CHECKABLE', 'OTHER']

# helper function to initialize document counts at zero
def get_experiment_docs(exp_id, sys_id=None):
    if sys_id == None:
        return {doc_id:0 for doc_id, e_id in doc_experiments.items() if e_id == exp_id }
    return {doc_id:0 for doc_id, e_id in doc_experiments.items() if e_id == exp_id and doc_systems[doc_id] == sys_id}
    

# Dicts to store each of the 4 levels (pre-populate all counts with zero)
ensemble_overall = {exp_id:get_experiment_docs(exp_id) for exp_id in experiments}
ensemble_type = {typ_id:{exp_id:get_experiment_docs(exp_id) for exp_id in experiments} for typ_id in error_types}
system_overall = {
    sys_id:{
        exp_id:get_experiment_docs(exp_id,sys_id) for exp_id in experiments
    } for sys_id in systems
}
system_type = {
    sys_id:{
        typ_id:{exp_id:get_experiment_docs(exp_id,sys_id) for exp_id in experiments} for typ_id in error_types
    } for sys_id in systems
}

# Iterate results and record against each of the above
for result in results:
    doc_id = result[0]
    sys_id = result[1]
    exp_id = result[2]
    typ_id = result[3]
    ensemble_overall[exp_id][doc_id] += 1
    ensemble_type[typ_id][exp_id][doc_id] += 1
    system_overall[sys_id][exp_id][doc_id] += 1
    system_type[sys_id][typ_id][exp_id][doc_id] += 1
    
# Helper function to account for means of empty lists
def get_mean(x):
    return np.mean(list(x.values()))

    
"""
    Ensemble-Overall measurements
"""

values = []
for exp_id in experiments:
    values.append(get_mean(ensemble_overall[exp_id]))
measurements['Ensemble-Overall'] = values


"""
    Ensemble-Type measurements
"""

for typ_id in error_types:
    key = f'Ensemble-{typ_id}'
    values = []
    print('')
    for exp_id in experiments:
        v = get_mean(ensemble_type[typ_id][exp_id])
        values.append(v)
        print(exp_id, typ_id, v)
        
    measurements[key] = values

"""
    System-Overall measurements
"""

for sys_id in systems:
    key = f'{sys_id}-Overall'
    values = []
    for exp_id in experiments:
        values.append(get_mean(system_overall[sys_id][exp_id]))
    measurements[key] = values

"""
    System-Type measurements
"""

for sys_id in systems:
    for typ_id in error_types:
        key = f'{sys_id}-{typ_id}'
        values = []
        for exp_id in experiments:
            values.append(get_mean(system_type[sys_id][typ_id][exp_id]))
        measurements[key] = values



A NAME 5.333333333333333
B NAME 5.256410256410256
C NAME 7.066666666666666

A NUMBER 8.857142857142858
B NUMBER 7.384615384615385
C NUMBER 7.466666666666667

A WORD 4.428571428571429
B WORD 6.17948717948718
C WORD 4.666666666666667

A CONTEXT 0.7619047619047619
B CONTEXT 0.8974358974358975
C CONTEXT 0.26666666666666666

A NOT_CHECKABLE 0.19047619047619047
B NOT_CHECKABLE 0.8461538461538461
C NOT_CHECKABLE 1.2666666666666666

A OTHER 0.047619047619047616
B OTHER 0.0
C OTHER 0.0


In [3]:
"""
  For the box plots
"""

import statistics

box_data = {sys_id:{exp_id:{} for exp_id in experiments} for sys_id in systems}

for result in results:
    doc_id = result[0]
    sys_id = result[1]
    exp_id = result[2]
    category = result[3]
    
    if doc_id not in box_data[sys_id][exp_id]:
        box_data[sys_id][exp_id][doc_id] = 0
        
    box_data[sys_id][exp_id][doc_id] += 1

# index median box_top box_bottom whisker_top whisker_bottom
for sys_id in systems:
    for i, exp_id in enumerate(experiments):
        exp_data = box_data[sys_id][exp_id]
        values = list(exp_data.values())
        x = np.quantile(values, [0.5,0.75,0.25,1,0])
        arr = [str(i)] + list([str(y) for y in x])
        print(' '.join(arr))

0 21.0 25.0 17.5 34.0 11.0
1 21.0 28.0 18.0 53.0 10.0
2 24.0 28.25 21.5 40.0 19.0
0 21.0 28.5 18.0 30.0 9.0
1 17.0 21.0 14.0 26.0 11.0
2 18.0 20.75 14.75 41.0 7.0
0 12.0 20.5 10.0 31.0 4.0
1 18.0 21.0 14.0 35.0 8.0
2 15.0 18.0 12.0 30.0 10.0


In [4]:
# -*- coding: utf-8 -*-
# Based on https://github.com/asbelz/coeff-var
"""This code computes the coefficient of variation (CV) and some other stats for small samples (indicated by the * added to CV) 
for a given set of measurements which are assumed to be for the same or similar object, using the same measurand. 
Stats are adjusted for small sample size. Paper ref: Belz, Popovic & Mille (2022) Quantified Reproducibility Assessment of NLP Results,
ACL'22.

In this self-contained version, the set of measurements on which CV is computed is assigned to the variable set_of_set_of_measurements
(see examples in code below).

The reproducibility stats reported in the output are: 
* the unbiased coefficient of variation
* the sample mean
* the unbiased sample standard deviation with 95% confidence intervals, estimated on the basis of the standard error of the unbiassed sample variance
* the sample size
* the percentage of measured valued within two standard deviations
* the percentage of measured valued within one standard deviation

Example narrative output:

The unbiased coefficient of variation is 1.5616560359100269 \
for a mean of 85.58285714285714 , \
unbiased sample standard deviation of 1.2904233075765223 with 95\% CI (0.4514829817654973, 2.1293636333875474) ,\
and a sample size of 7 . \
100.0 % of measured values fall within two standard deviations. \
71.429 % of measured values fall within one standard deviation. 

NOTE:
* CV assumes all measurements are positive; if they're not, shift measurement scale to start at 0
* for fair comparison across studies, measurements on a scale that doesn't start at 0 need to be shifted to a scale that does start at 0 

KNOWN ISSUES:

none
"""

import math
import numpy as np
from scipy.stats import t

def format_result(key, measurements, cf):
    arr = [key] + [str(round(m,2)) for m in measurements] + [str(round(cf,2))]
    return ' & '.join(arr)



for k, set_of_measurements in measurements.items():
  assert len(set_of_measurements) == 3
  if len(set_of_measurements) < 2:
    print(set_of_measurements, ": set of measurements is smaller than 2")
    continue

  sample_mean = np.mean(set_of_measurements)
  if sample_mean <= 0:
    sample_mean = -1
    small_sample_coefficient_of_variation = -1
  else:
    sample_size = len(set_of_measurements)
    degrees_of_freedom = sample_size-1
    sum_of_squared_differences = np.sum(np.square(sample_mean-set_of_measurements))

    # unbiassed sample variance s^2
    unbiassed_sample_variance = sum_of_squared_differences/degrees_of_freedom
    # corrected sample standard deviation s
    corrected_sample_standard_deviation = np.sqrt(unbiassed_sample_variance)
    # Gamma(N/2)
    gamma_N_over_2 = math.gamma(sample_size/2)
    # Gamma((N-1)/2)
    gamma_df_over_2 = math.gamma(degrees_of_freedom/2)
    # c_4(N)
    c_4_N = math.sqrt(2/degrees_of_freedom)*gamma_N_over_2/gamma_df_over_2
    # unbiassed sample std dev s/c_4
    unbiassed_sample_std_dev_s_c_4 = corrected_sample_standard_deviation/c_4_N
    # standard error of the unbiassed sample variance (assumes normally distributed population)
    standard_error_of_unbiassed_sample_variance = unbiassed_sample_variance*np.sqrt(2/degrees_of_freedom)
    # estimated std err of std dev based on std err of unbiassed sample variance
    est_SE_of_SD_based_on_SE_of_unbiassed_sample_variance = standard_error_of_unbiassed_sample_variance/(2*unbiassed_sample_std_dev_s_c_4)

    # COEFFICIENT OF VARIATION CV
    coefficient_of_variation = (unbiassed_sample_std_dev_s_c_4/sample_mean)*100
    # SMALL SAMPLE CORRECTED COEFFICIENT OF VARIATION CV*
    small_sample_coefficient_of_variation = (1+(1/(4*sample_size)))*coefficient_of_variation

    # compute percentage of measured values within 1 and 2 standard deviations from the mean
    # initialise counts
    count_within_1_sd = 0
    count_within_2_sd = 0
    # for each measured value
    for m in set_of_measurements:
      # if it's within two std devs, increment count_within_2_sd
      if np.abs(m-sample_mean) < 2*unbiassed_sample_std_dev_s_c_4:
        count_within_2_sd += 1
        #if it's also within one std devs, increment count_within_1_sd
        if np.abs(m-sample_mean) < unbiassed_sample_std_dev_s_c_4:
          count_within_1_sd += 1
        
  # Print results for the latex tables (all values are calculated then final values rounded)
  arr = [k] + [('%.2f' % m) for m in set_of_measurements] + ['%.2f' % small_sample_coefficient_of_variation]
  print(' & '.join(arr) + ' \\\\')

  # report results as described in code description above
#   print(f"The unbiased coefficient of variation ({k}) is",small_sample_coefficient_of_variation)
#   print("for a mean of",sample_mean,", ")
#   print("unbiased sample standard deviation of",unbiassed_sample_std_dev_s_c_4,", with 95\% CI",t.interval(0.95, degrees_of_freedom, loc=unbiassed_sample_std_dev_s_c_4, scale=est_SE_of_SD_based_on_SE_of_unbiassed_sample_variance),",")
#   print("and a sample size of",sample_size,".")
#   print(count_within_2_sd/sample_size*100,"% of measured values fall within two standard deviations.")
#   print(round(count_within_1_sd/sample_size*100, 3),"% of measured values fall within one standard deviation.", )


Ensemble-Overall & 19.62 & 20.56 & 20.73 & 3.61 \\
Ensemble-NAME & 5.33 & 5.26 & 7.07 & 21.26 \\
Ensemble-NUMBER & 8.86 & 7.38 & 7.47 & 12.80 \\
Ensemble-WORD & 4.43 & 6.18 & 4.67 & 22.80 \\
Ensemble-CONTEXT & 0.76 & 0.90 & 0.27 & 63.22 \\
Ensemble-NOT_CHECKABLE & 0.19 & 0.85 & 1.27 & 86.35 \\
Ensemble-OTHER & 0.05 & 0.00 & 0.00 & 211.73 \\
conditional_copy-Overall & 21.57 & 25.54 & 26.60 & 13.19 \\
document_plan-Overall & 21.86 & 17.77 & 18.90 & 13.23 \\
hierarchical_encoder-Overall & 15.43 & 18.38 & 16.70 & 10.77 \\
conditional_copy-NAME & 5.57 & 6.00 & 7.80 & 22.39 \\
conditional_copy-NUMBER & 9.29 & 10.92 & 11.40 & 12.87 \\
conditional_copy-WORD & 5.86 & 7.15 & 6.00 & 13.72 \\
conditional_copy-CONTEXT & 0.43 & 0.23 & 0.10 & 79.89 \\
conditional_copy-NOT_CHECKABLE & 0.43 & 1.23 & 1.30 & 60.02 \\
conditional_copy-OTHER & 0.00 & 0.00 & 0.00 & -1.00 \\
document_plan-NAME & 5.71 & 5.08 & 6.40 & 14.12 \\
document_plan-NUMBER & 11.14 & 6.15 & 7.00 & 40.30 \\
document_plan-WORD & 4.43 & 5.